In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
import seaborn.palettes
import seaborn.utils
import sys
from census import Census
# from us import states
import http.client, urllib.request, urllib.parse, urllib.error, base64
#import config
import quickstart

In [119]:
# READ CENSUS DATA
census_tract= pd.read_csv('acs_2019onlynew.csv')
census_tract.head().columns

Index(['Unnamed: 0', 'geofips', 'totpop', 'popdens', 'areasqmile', 'males',
       'females', 'pop18under', 'pop18_34', 'pop35_64', 'pop65plus',
       'pophispanic', 'popnhwhite', 'popnhblack', 'popnhaian', 'popnhasian',
       'popnhhawainpi', 'popnhother', 'popnhtwoplus', 'housing',
       'housingoccupied', 'renterhousing', 'oc_halfless', 'oc_half_1',
       'oc_1_onehalf', 'oc_onehalf_2', 'oc_2plus', 'roc_halfless',
       'roc_half_1', 'roc_1_onehalf', 'roc_onehalf_2', 'roc_2plus',
       'medianyearbuilt', 'mediangrossrent', 'medianhhincome', 'gini', 'year',
       'pctmale', 'pct18under', 'pct18_34', 'pct35_64', 'pct65plus',
       'pcthispanic', 'pctnhwhite', 'pctnhblack', 'pctnhaian', 'pctnhasian',
       'pctnhhawainpi', 'pctnhother', 'pctnhtwoplus', 'pctownerocch',
       'pctrentocch', 'crowding', 'crowdingrenter', 'Geo_QName',
       'housingoccupied2', 'hhfgas', 'hhfelectricity', 'hhfoilkerosene',
       'hhfcoalwood', 'hhfsolarenergy', 'hhfotherfuel', 'hhfnofuel',
     

In [26]:
# census_tract= pd.read_csv('acs_2019only.csv').drop("Unnamed: 0", axis = 1).rename({"BoroCT2010":"boroct2010"}, axis = 1)

# labels_to_drop = ['pct18under',
#        'pct18_34', 'pct35_64', 'pct65plus', 'pcthispanic', 'pctnhwhite',
#        'pctnhblack', 'pctnhaian', 'pctnhasian', 'pctnhhawainpi', 'pctnhother',
#        'pctnhtwoplus', 'pctownerocch', 'pctrentocch']
# census_tract = census_tract.drop(labels_to_drop, axis = 1)
# # census_tract = census_tract[census_tract['year'] == 2019]
# census_tract = census_tract.fillna(0)

# # make region key the same as crosswalk file
# def get_region(x):
#     y = int(str(x)[:5])
#     z = str(x)[5:]
#     match y:
#         case (36005):
#             return int("2" + z)
#         case (36047):
#             return int("3" + z)
#         case (36061):
#             return int("1" + z)
#         case (36081):
#             return int("4" + z)
#         case (36085):
#             return int("5" + z)
# census_tract["geofips"] = census_tract["geofips"].apply(lambda x : get_region(x))

# census_tract

,geofips,totpop,popdens,areasqmile,males,females,pop18under,pop18_34,pop35_64,pop65plus,...,roc_onehalf_2,roc_2plus,medianyearbuilt,mediangrossrent,medianhhincome,gini,year,pctmale,crowding,crowdingrenter
21721,2000100,6864,10599.5500,0.647574,6345,519,103,3322,3392,47,...,0,0,0.0,0.0,0.0,0.0000,2019,92.438811,0.000000,0.000000
21722,2000200,4532,24481.1900,0.185122,2193,2339,1134,862,1605,931,...,31,11,1957.0,1559.0,51100.0,0.4460,2019,48.389232,11.997019,18.181818
21723,2000400,5516,18537.1400,0.297565,2810,2706,1071,1319,2362,764,...,21,0,2000.0,1314.0,78409.0,0.4141,2019,50.942712,1.860921,4.906542
21724,2001600,5825,31101.4900,0.187290,2422,3403,1546,1399,1978,902,...,46,45,1973.0,1030.0,34093.0,0.5881,2019,41.579399,11.415525,12.514484
21725,2001900,3141,4877.1520,0.644023,1583,1558,898,1052,1111,80,...,18,0,1965.0,1344.0,45156.0,0.5076,2019,50.397962,9.638554,10.777202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23883,5030302,6702,19100.8000,0.350875,3480,3222,1889,1627,2507,679,...,49,0,1984.0,1945.0,85660.0,0.3618,2019,51.924799,5.199394,10.404624
23884,5031901,2816,18795.0200,0.149827,1294,1522,686,904,929,297,...,0,0,1958.0,351.0,21250.0,0.6332,2019,45.951705,8.468126,5.209841
23885,5031902,5343,20039.8600,0.266619,2414,2929,1799,1417,1720,407,...,5,12,1977.0,761.0,46118.0,0.5606,2019,45.180610,10.603291,14.022989
23886,5032300,1073,718.2481,1.493913,553,520,225,195,508,145,...,0,0,1991.0,1621.0,65089.0,0.5363,2019,51.537745,1.773836,0.000000


In [40]:
# census_tract_fuel = pd.read_csv('acs_2019_fuel_mobility.csv').drop(["Unnamed: 0"], axis=1)
# census_tract_fuel["Geo_FIPS"] = census_tract_fuel["Geo_FIPS"].apply(lambda x : get_region(x))
# census_tract_fuel.iloc[:,1:] = census_tract_fuel.iloc[:,1:].apply(lambda x : (x/100)*)
# census_tract_fuel.fillna(0)

,Geo_FIPS,pcthgas,pcthelectricity,pcthoilker,pcthcoal,pcthsolar,pcthotherf,pcthnofuel,pctsamehouse,pctmvdsamecounty,pctmvdsdiffcounty,pctmvdsdiffstate,pctmvdabroad,pop1yrplusinhh,pctpoprenterhh
0,2000100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,2000200,0.750373,0.115499,0.134128,0.000000,0.000000,0.000000,0.000000,0.979176,0.006016,0.005553,0.009255,0.000000,45.15,0.002972
2,2000400,0.685113,0.167483,0.140548,0.000000,0.000000,0.000000,0.006856,1.000000,0.000000,0.000000,0.000000,0.000000,54.50,0.003747
3,2001600,0.403349,0.129883,0.424658,0.000000,0.000000,0.000000,0.042111,0.961442,0.020217,0.010421,0.002709,0.005211,55.61,0.003544
4,2001900,0.438369,0.230769,0.259500,0.000000,0.000000,0.040778,0.030584,0.888350,0.033539,0.052515,0.022507,0.003089,26.55,0.004064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2162,5030302,0.918728,0.048965,0.021706,0.010601,0.000000,0.000000,0.000000,0.991706,0.008294,0.000000,0.000000,0.000000,65.94,0.003004
2163,5031901,0.927688,0.017127,0.055186,0.000000,0.000000,0.000000,0.000000,0.987780,0.012220,0.000000,0.000000,0.000000,27.92,0.003764
2164,5031902,0.897623,0.056063,0.021328,0.000000,0.018282,0.000000,0.006703,0.975383,0.015335,0.002825,0.000000,0.006457,52.55,0.003123
2165,5032300,0.889135,0.101996,0.008869,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,10.73,0.004203


In [120]:
crosswalk = pd.read_csv('ct_box_xwalk1.csv')
crosswalk = crosswalk.sort_values(by="borobox")

In [121]:
columnNames = ['borobox', 'count', 'totpop', 'popdens', 'areasqmile', 'males', 'females',
       'pop18under', 'pop18_34', 'pop35_64', 'pop65plus', 'pophispanic',
       'popnhwhite', 'popnhblack', 'popnhaian', 'popnhasian', 'popnhhawainpi',
       'popnhother', 'popnhtwoplus', 'housing', 'housingoccupied',
       'renterhousing', 'oc_halfless', 'oc_half_1', 'oc_1_onehalf',
       'oc_onehalf_2', 'oc_2plus', 'roc_halfless', 'roc_half_1',
       'roc_1_onehalf', 'roc_onehalf_2', 'roc_2plus', 'medianyearbuilt',
       'mediangrossrent', 'medianhhincome', 'gini', 'year', 'crowdingrenter', 'crowding', "housingoccupied2",
       "hhfgas","hhfelectricity","hhfoilkerosene","hhfcoalwood","hhfsolarenergy","hhfotherfuel","hhfnofuel"
       ,"pop1yrplusinhh","poprenterhu","popsamehouse","popmvdsamecounty","popmvddiffcounty","popmvdstate","popmvdabroad"]
weighted_borobox = pd.DataFrame(columns = columnNames)
numRuns = 0

for index, row in crosswalk.iterrows():
    box = row["borobox"]
    weight = row["wgt"]
    try:
        tract = int(row["boroct2010"])
    except:
        continue

    try: 
        
        row_census = census_tract.loc[census_tract['geofips_recode'] == tract].iloc[-1]

        # ------------ in order to test with smaller datset, uncomment the below three lines ---------
        # numRuns += 1
        # if numRuns > 10:
        #     break
        # ------------ end test section ---------------

# ---------add new row with borobox specified if row does not exist, otherwise edit existing row
        try:            # find existing row
            row_weighted = weighted_borobox.loc[weighted_borobox["borobox"] == box] # make copy of existing data to act on it
            row_weighted.at[0,'count'] = row_weighted.at[0,'count'] + 1
            for i, column in enumerate(columnNames):
                if i<2:
                    continue
                # elif (i<32):
                #     row_weighted.at[0,column] = (row_weighted.at[0,column] + (row_census[column] * weight))
                else:
                    # row_weighted.at[0,column] = row_weighted.at[0,column] + row_census[column]
                    row_weighted.at[0,column] = (row_weighted.at[0,column] + (row_census[column] * weight))
            weighted_borobox.loc[weighted_borobox["borobox"] == box] = row_weighted # paste in finalized new data

        except:         # create new row
            # create a column that initiates at one, if column is accessed again, increment. add up some columns that require averages. at end, divide to find target numbers.
            row_weighted = pd.DataFrame(columns = columnNames)
            row_weighted.at[0,'count'] = 1
            row_weighted.at[0,'borobox'] = box
            for i, column in enumerate(columnNames):
                if i<2:
                    continue
                # elif (i<32):
                #     row_weighted.at[0,column] = (row_census[column] * weight)
                else:
                    # row_weighted.at[0,column] = row_census[column]
                    row_weighted.at[0,column] = (row_census[column] * weight)
            weighted_borobox = pd.concat([weighted_borobox,row_weighted])
    except:
        print("error number: ")
        continue

In [122]:
for index, row in weighted_borobox.iterrows():
    instances = int(row["count"])
    row.at['medianyearbuilt'] = (row.at['medianyearbuilt']/instances)
    row.at['mediangrossrent'] = (row.at['mediangrossrent']/instances)
    row.at['medianhhincome'] = (row.at['medianhhincome']/instances)
    row.at['gini'] = (row.at['gini']/instances)
    row.at['year'] = (row.at['year']/instances)

In [123]:
for index_label, row_series in weighted_borobox.iterrows():
       # print(index_label)
       # if indexNum > 10:
       #        break
       totalPop = row_series.at['totpop']
       weighted_borobox.loc[index_label,'pctmale'] = ((row_series.at['males']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pct18under'] = ((row_series.at['pop18under']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pct18_34'] = ((row_series.at['pop18_34']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pct35_64'] = ((row_series.at['pop35_64']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pct65plus'] = ((row_series.at['pop65plus']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pcthispanic'] = ((row_series.at['pophispanic']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhwhite'] = ((row_series.at['popnhwhite']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhblack'] = ((row_series.at['popnhblack']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhaian'] = ((row_series.at['popnhaian']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhasian'] = ((row_series.at['popnhasian']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhhawainpi'] = ((row_series.at['popnhhawainpi']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhother'] = ((row_series.at['popnhwhite']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhtwoplus'] = ((row_series.at['popnhtwoplus']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctnhaian'] = ((row_series.at['popnhaian']/totalPop) * 100)
       weighted_borobox.loc[index_label,'pctownerocch'] = (row_series.at['housingoccupied'] / row_series.at['housing'])*100
       weighted_borobox.loc[index_label,'pctrentocch'] = (row_series.at['renterhousing'] / row_series.at['housing'])*100
       weighted_borobox.loc[index_label, 'crowding']= ((row_series.at['oc_1_onehalf']+row_series.at['oc_onehalf_2']+row_series.at['oc_2plus']) / row_series.at['housing'])*100
       weighted_borobox.loc[index_label, 'crowdingrenter']= ((row_series.at['roc_1_onehalf']+row_series.at['roc_onehalf_2']+row_series.at['roc_2plus']) / row_series.at['renterhousing'])*100
       weighted_borobox.loc[index_label,'pctsamehouse']= (row_series.at['popsamehouse'] / row_series.at['poprenterhu'])*100
       weighted_borobox.loc[index_label,'pctmvdsamecounty']= (row_series.at['popmvdsamecounty'] / row_series.at['poprenterhu'])*100
       weighted_borobox.loc[index_label,'pctmvdsdiffcounty']= (row_series.at['popmvddiffcounty'] / row_series.at['poprenterhu'])*100
       weighted_borobox.loc[index_label,'pctmvdsdiffstate']= (row_series.at['popmvdstate'] / row_series.at['poprenterhu'])*100
       weighted_borobox.loc[index_label,'pctmvdabroad']= (row_series.at['popmvdabroad'] / row_series.at['poprenterhu'])*100
       weighted_borobox.loc[index_label,'pctpoprenterhh']= (row_series.at['poprenterhu'])/(row_series.at['pop1yrplusinhh'])


/var/folders/nx/v953nxd50l77lmpgtc17c1m00000gn/T/ipykernel_22655/1209933034.py:23: RuntimeWarning: invalid value encountered in double_scalars
  weighted_borobox.loc[index_label, 'crowdingrenter']= ((row_series.at['roc_1_onehalf']+row_series.at['roc_onehalf_2']+row_series.at['roc_2plus']) / row_series.at['renterhousing'])*100
/var/folders/nx/v953nxd50l77lmpgtc17c1m00000gn/T/ipykernel_22655/1209933034.py:24: RuntimeWarning: invalid value encountered in double_scalars
  weighted_borobox.loc[index_label,'pctsamehouse']= (row_series.at['popsamehouse'] / row_series.at['poprenterhu'])*100
/var/folders/nx/v953nxd50l77lmpgtc17c1m00000gn/T/ipykernel_22655/1209933034.py:25: RuntimeWarning: invalid value encountered in double_scalars
  weighted_borobox.loc[index_label,'pctmvdsamecounty']= (row_series.at['popmvdsamecounty'] / row_series.at['poprenterhu'])*100
/var/folders/nx/v953nxd50l77lmpgtc17c1m00000gn/T/ipykernel_22655/1209933034.py:26: RuntimeWarning: invalid value encountered in double_scala

In [124]:

median_rent = weighted_borobox["mediangrossrent"].median()
median_income = weighted_borobox["medianhhincome"].median()
median_year = weighted_borobox["medianyearbuilt"].median()

m = weighted_borobox.loc[weighted_borobox["mediangrossrent"].isna()]
for index_label, row_series in m.iterrows():
    weighted_borobox.loc[index_label,'mediangrossrent'] = median_rent
m2 = weighted_borobox.loc[weighted_borobox["medianhhincome"].isna()]
for index_label, row_series in m2.iterrows():
    weighted_borobox.loc[index_label,'medianhhincome'] = median_income
m3 = weighted_borobox.loc[weighted_borobox["medianyearbuilt"].isna()]
for index_label, row_series in m3.iterrows():
    weighted_borobox.loc[index_label,'medianyearbuilt'] = median_year

In [125]:
weighted_borobox.isna().sum(axis = 0).sort_values(ascending=False).head(15)

gini                9
borobox             0
pop1yrplusinhh      0
pctmale             0
popmvdabroad        0
popmvdstate         0
popmvddiffcounty    0
popmvdsamecounty    0
popsamehouse        0
poprenterhu         0
hhfnofuel           0
crowding            0
hhfotherfuel        0
hhfsolarenergy      0
hhfcoalwood         0
dtype: int64

In [126]:
weighted_borobox.to_csv('weighted_borobox.csv')